In [73]:
import pandas as pd
import numpy as np

Strompreis=pd.read_csv("Daten entsoe/Gro_handelspreise_202311010000_202502010000_Stunde.csv",sep=';')

In [74]:
Strompreis['Datum']=pd.to_datetime(Strompreis['Datum von'],format='%d.%m.%Y %H:%S')
Strompreis=Strompreis.sort_values('Datum',kind='stable')
Strompreis['Datum']=pd.to_datetime(Strompreis['Datum von'],format='%d.%m.%Y %H:%S')
Strompreis['Datum']=Strompreis['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Strompreis['Date']=Strompreis['Datum'].dt.tz_convert('UTC')
Strompreis=Strompreis.set_index('Date')

In [81]:
datei="Daten entsoe/Physikalischer_Stromfluss_202311010000_202502010000_Viertelstunde.csv"
dtype={'Datum von':"string",'Datum bis':"string"
       ,'Nettoexport [MWh] Originalauflösungen':np.float64
       ,'Niederlande (Export) [MWh] Originalauflösungen':np.float64
       ,'Niederlande (Import) [MWh] Originalauflösungen':np.float64
       ,'Schweiz (Export) [MWh] Originalauflösungen':np.float64
       ,'Schweiz (Import) [MWh] Originalauflösungen':np.float64
       ,'Dänemark (Export) [MWh] Originalauflösungen':np.float64
       ,'Dänemark (Import) [MWh] Originalauflösungen':np.float64
       ,'Tschechien (Export) [MWh] Originalauflösungen':np.float64
       ,'Tschechien (Import) [MWh] Originalauflösungen':np.float64
       ,'Luxemburg (Export) [MWh] Originalauflösungen':np.float64
       ,'Luxemburg (Import) [MWh] Originalauflösungen':np.float64
       ,'Schweden (Export) [MWh] Originalauflösungen':np.float64
       ,'Schweden (Import) [MWh] Originalauflösungen':np.float64
       ,'Österreich (Export) [MWh] Originalauflösungen':np.float64
       ,'Österreich (Import) [MWh] Originalauflösungen':np.float64
       ,'Frankreich (Export) [MWh] Originalauflösungen':np.float64
       ,'Frankreich (Import) [MWh] Originalauflösungen':np.float64
       ,'Polen (Export) [MWh] Originalauflösungen':np.float64
       ,'Polen (Import) [MWh] Originalauflösungen':np.float64
       ,'Norwegen (Export) [MWh] Originalauflösungen':np.float64
       ,'Norwegen (Import) [MWh] Originalauflösungen':np.float64
       ,'Belgien (Export) [MWh] Originalauflösungen':np.float64
       ,'Belgien (Import) [MWh] Originalauflösungen':np.float64}


#Stromfluss=pd.read_csv("Daten entsoe/Physikalischer_Stromfluss_202311010000_202502010000_Viertelstunde.csv",sep=';', decimal=",", thousands=".",dtype=dtype,on_bad_lines='skip')
Stromfluss=pd.read_csv(datei,sep=';', decimal=",", thousands=".",header=0,on_bad_lines='skip',low_memory=False)

In [82]:
Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%S')
#Stromfluss=Stromfluss.sort_values('Datum',kind='stable')
Stromfluss['Datum']=pd.to_datetime(Stromfluss['Datum von'],format='%d.%m.%Y %H:%S')
Stromfluss['Datum']=Stromfluss['Datum'].dt.tz_localize('Europe/Berlin',ambiguous='infer')
Stromfluss['Date']=Stromfluss['Datum'].dt.tz_convert('UTC')
#Stromfluss=Stromfluss.set_index('Date')

In [83]:
Stromfluss.groupby(Stromfluss['Date'].dt.floor('h'))['Nettoexport [MWh] Originalauflösungen'].agg(['sum'])

,sum
Date,
2023-10-31 23:00:00+00:00,"-2.565,00-2.464,00-2.501,00-2.665,50"
2023-11-01 00:00:00+00:00,"-2.298,00-2.341,50-2.339,75-2.374,25"
2023-11-01 01:00:00+00:00,"-2.387,50-2.378,50-2.434,00-2.422,25"
2023-11-01 02:00:00+00:00,"-2.346,75-2.404,00-2.422,50-2.485,00"
2023-11-01 03:00:00+00:00,"-2.408,75-2.454,25-2.473,50-2.507,75"
...,...
2025-01-31 18:00:00+00:00,----
2025-01-31 19:00:00+00:00,----
2025-01-31 20:00:00+00:00,----


In [84]:
Stromfluss

,Datum von,Datum bis,Nettoexport [MWh] Originalauflösungen,Niederlande (Export) [MWh] Originalauflösungen,Niederlande (Import) [MWh] Originalauflösungen,Schweiz (Export) [MWh] Originalauflösungen,Schweiz (Import) [MWh] Originalauflösungen,Dänemark (Export) [MWh] Originalauflösungen,Dänemark (Import) [MWh] Originalauflösungen,Tschechien (Export) [MWh] Originalauflösungen,...,Frankreich (Export) [MWh] Originalauflösungen,Frankreich (Import) [MWh] Originalauflösungen,Polen (Export) [MWh] Originalauflösungen,Polen (Import) [MWh] Originalauflösungen,Norwegen (Export) [MWh] Originalauflösungen,Norwegen (Import) [MWh] Originalauflösungen,Belgien (Export) [MWh] Originalauflösungen,Belgien (Import) [MWh] Originalauflösungen,Datum,Date
0,01.11.2023 00:00,01.11.2023 00:15,"-2.565,00","221,25","0,00","37,50","-262,50","0,00","-192,75","0,00",...,"0,00","-939,50","0,00","-278,00","0,00","-242,50","0,00","-68,25",2023-11-01 00:00:00+01:00,2023-10-31 23:00:00+00:00
1,01.11.2023 00:15,01.11.2023 00:30,"-2.464,00","235,00","0,00","52,50","-264,50","0,00","-173,25","0,00",...,"0,00","-939,50","0,00","-278,00","0,00","-222,75","0,00","-71,25",2023-11-01 00:00:15+01:00,2023-10-31 23:00:15+00:00
2,01.11.2023 00:30,01.11.2023 00:45,"-2.501,00","233,75","0,00","53,50","-305,75","0,00","-182,25","0,00",...,"0,00","-939,50","0,00","-278,00","0,00","-223,00","0,00","-71,25",2023-11-01 00:00:30+01:00,2023-10-31 23:00:30+00:00
3,01.11.2023 00:45,01.11.2023 01:00,"-2.665,50","163,25","0,00","58,50","-389,75","0,00","-171,50","0,00",...,"0,00","-939,50","0,00","-278,00","0,00","-200,25","0,00","-80,25",2023-11-01 00:00:45+01:00,2023-10-31 23:00:45+00:00
4,01.11.2023 01:00,01.11.2023 01:15,"-2.298,00","162,25","0,00","51,00","-361,50","0,00","-138,50","0,00",...,"0,00","-916,50","0,00","-244,75","0,00","-110,25","0,00","-174,25",2023-11-01 01:00:00+01:00,2023-11-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43963,31.01.2025 22:45,31.01.2025 23:00,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,2025-01-31 22:00:45+01:00,2025-01-31 21:00:45+00:00
43964,31.01.2025 23:00,31.01.2025 23:15,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,2025-01-31 23:00:00+01:00,2025-01-31 22:00:00+00:00
43965,31.01.2025 23:15,31.01.2025 23:30,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,2025-01-31 23:00:15+01:00,2025-01-31 22:00:15+00:00
43966,31.01.2025 23:30,31.01.2025 23:45,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,2025-01-31 23:00:30+01:00,2025-01-31 22:00:30+00:00


In [54]:
Strom

DatetimeIndex(['2023-10-31 23:00:00+00:00', '2023-10-31 23:00:15+00:00',
               '2023-10-31 23:00:30+00:00', '2023-10-31 23:00:45+00:00',
               '2023-11-01 00:00:00+00:00', '2023-11-01 00:00:15+00:00',
               '2023-11-01 00:00:30+00:00', '2023-11-01 00:00:45+00:00',
               '2023-11-01 01:00:00+00:00', '2023-11-01 01:00:15+00:00',
               ...
               '2025-01-31 20:00:30+00:00', '2025-01-31 20:00:45+00:00',
               '2025-01-31 21:00:00+00:00', '2025-01-31 21:00:15+00:00',
               '2025-01-31 21:00:30+00:00', '2025-01-31 21:00:45+00:00',
               '2025-01-31 22:00:00+00:00', '2025-01-31 22:00:15+00:00',
               '2025-01-31 22:00:30+00:00', '2025-01-31 22:00:45+00:00'],
              dtype='datetime64[ns, UTC]', name='Date', length=43968, freq=None)

In [52]:
Stromfluss[Stromfluss['Datum'].dt.day_of_year==301]

,Datum von,Datum bis,Nettoexport [MWh] Originalauflösungen,Niederlande (Export) [MWh] Originalauflösungen,Niederlande (Import) [MWh] Originalauflösungen,Schweiz (Export) [MWh] Originalauflösungen,Schweiz (Import) [MWh] Originalauflösungen,Dänemark (Export) [MWh] Originalauflösungen,Dänemark (Import) [MWh] Originalauflösungen,Tschechien (Export) [MWh] Originalauflösungen,...,Österreich (Import) [MWh] Originalauflösungen,Frankreich (Export) [MWh] Originalauflösungen,Frankreich (Import) [MWh] Originalauflösungen,Polen (Export) [MWh] Originalauflösungen,Polen (Import) [MWh] Originalauflösungen,Norwegen (Export) [MWh] Originalauflösungen,Norwegen (Import) [MWh] Originalauflösungen,Belgien (Export) [MWh] Originalauflösungen,Belgien (Import) [MWh] Originalauflösungen,Datum
Date,,,,,,,,,,,,,,,,,,,,,
2024-10-26 22:00:00+00:00,27.10.2024 00:00,27.10.2024 00:15,"284,50","1.090,50","0,00","107,00","-67,25","0,00","-549,50","374,00",...,"-61,75","0,00","-766,00","449,25","0,00","0,00","-190,50","0,00","-161,75",2024-10-27 00:00:00+02:00
2024-10-26 22:00:15+00:00,27.10.2024 00:15,27.10.2024 00:30,"209,75","1.105,75","0,00","123,50","-84,00","0,00","-594,00","338,25",...,"-76,75","0,00","-766,00","468,50","0,00","0,00","-185,50","0,00","-165,25",2024-10-27 00:00:15+02:00
2024-10-26 22:00:30+00:00,27.10.2024 00:30,27.10.2024 00:45,"148,25","1.156,75","0,00","106,75","-114,00","0,00","-599,25","331,75",...,"-75,50","0,00","-766,00","462,00","0,00","0,00","-185,50","0,00","-212,50",2024-10-27 00:00:30+02:00
2024-10-26 22:00:45+00:00,27.10.2024 00:45,27.10.2024 01:00,"104,00","1.172,25","0,00","107,50","-116,75","0,00","-612,00","319,50",...,"-66,00","0,00","-766,00","453,00","0,00","0,00","-183,75","0,00","-228,00",2024-10-27 00:00:45+02:00
2024-10-26 23:00:00+00:00,27.10.2024 01:00,27.10.2024 01:15,"224,00","1.082,00","0,00","114,00","-81,00","0,00","-542,50","345,00",...,"-56,25","0,00","-813,50","431,00","0,00","0,00","-177,00","0,00","-152,75",2024-10-27 01:00:00+02:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-27 21:00:45+00:00,27.10.2024 22:45,27.10.2024 23:00,"-2.621,00","266,75","-0,50","74,75","-447,75","0,00","-848,50","122,25",...,"-193,00","0,00","-905,00","73,25","0,00","0,00","-347,00","0,00","-236,25",2024-10-27 22:00:45+01:00
2024-10-27 22:00:00+00:00,27.10.2024 23:00,27.10.2024 23:15,"-2.389,50","280,25","0,00","82,25","-347,25","0,00","-878,00","142,50",...,"-139,25","0,00","-924,75","47,25","-1,50","0,00","-330,25","0,00","-141,50",2024-10-27 23:00:00+01:00
2024-10-27 22:00:15+00:00,27.10.2024 23:15,27.10.2024 23:30,"-2.512,75","307,25","0,00","81,00","-344,50","0,00","-852,50","113,75",...,"-166,75","0,00","-924,75","8,75","-18,25","0,00","-326,00","0,00","-167,50",2024-10-27 23:00:15+01:00


In [30]:
dir(pd.to_datetime('2024-01-01'))

['__add__',
 '__array_priority__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__radd__',
 '__reduce__',
 '__reduce_cython__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__setstate__',
 '__setstate_cython__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__weakref__',
 '_creso',
 '_date_repr',
 '_from_dt64',
 '_from_value_and_reso',
 '_repr_base',
 '_round',
 '_time_repr',
 '_value',
 'as_unit',
 'asm8',
 'astimezone',
 'ceil',
 'combine',
 'ctime',
 'date',
 'day',
 'day_name',
 'day_of_week',
 'day_of_year',
 'dayofweek',
 'dayofyear',
 'days_in_month',
 'daysinmonth',
 'dst',
 'floor',
 'fold',
 'fromisocalendar',
 'fromisoformat',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'is_leap_year',
 'is_month_end',
 'is_mo